<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.py
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [2]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox

import che
import numpy as np
from scipy.optimize import root

In [3]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [4]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
c=DotDict()
c.Ftot=10 # Total Feed moles
c.Fz = np.array([1/3, 1/3, 1/3]) # Equimolar feed composition
c.FT = 450 # Feed temperature
c.flashP= 101325 # Flash drum pressure

c.Vy = Comp(c.Fz) # Guess vapor/liquid composition equal to feed
c.Lx = Comp(c.Fz) # Comp - constrains mole fractions to behave like mole fractions!
c.flashT = Range(360, 273.15, c.FT)  # Guess and bounds for flash temperature
c.Vtot = Range(c.Ftot/2, 0., c.Ftot)  # Guess half of feed in vapor
c.Ltot = Range(c.Ftot/2, 0., c.Ftot)

In [16]:
wrap, x, d2, xtod, xtodunk = dtox(c)

In [17]:
d2

{'FT': 450,
 'Ftot': 10,
 'Fz': array([0.33333333, 0.33333333, 0.33333333]),
 'Ltot': array([5.]),
 'Lx': array([0.33333333, 0.33333333, 0.33333333]),
 'Vtot': array([5.]),
 'Vy': array([0.33333333, 0.33333333, 0.33333333]),
 'flashP': 101325,
 'flashT': array([360.])}

In [18]:
def eqs(d):
    r=DotDict()
    V = d.Vy * d.Vtot # Moles of each component = mole fractions * total moles
    L = d.Lx * d.Ltot
    F = d.Fz * d.Ftot
    r.mass_balance = F - V - L # Mass balance for each component (vectors!)

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    FH = p.Hl(nL=F, T=d.FT)
    VH = p.Hv(nV=V, T=d.flashT)
    LH = p.Hl(nL=L, T=d.flashT)
    r.energy_balance = (FH - VH - LH)

    # Raoults with NRTL activity coefficient correction.  One-liner!
    fugL = d.Lx  * p.NRTL_gamma(d.Lx,d.flashT)* p.Pvap(d.flashT)
    fugV = d.Vy * d.flashP
    r.VLE = fugL - fugV
    return np.concatenate([r.mass_balance, np.atleast_1d(r.energy_balance), r.VLE]), r 


In [19]:
eqs_wrapped = wrap(eqs)

In [20]:
sol=root(eqs_wrapped, x)
sol

    fjac: array([[-2.93229927e-05,  1.53807739e-05,  1.39422188e-05,
        -6.47951742e-01, -6.22484539e-01,  2.98554402e-01,
         3.21771358e-01],
       [-1.26264656e-05, -1.90380746e-05,  3.16645406e-05,
         4.33720518e-01, -2.83920771e-01, -4.40580740e-01,
         7.32914809e-01],
       [ 4.23239467e-05, -8.24820605e-06, -3.40757409e-05,
         6.25454608e-01, -4.73857215e-01,  5.86318559e-01,
        -2.01237217e-01],
       [ 6.88535283e-04,  2.57839041e-04, -9.46374329e-04,
        -2.91819235e-02, -5.54401384e-01, -6.10724116e-01,
        -5.64625662e-01],
       [-5.81789459e-01, -2.05230302e-01,  7.87019764e-01,
        -3.42414631e-06, -6.62166607e-04, -6.94160961e-04,
        -7.21128061e-04],
       [-6.06648947e-01,  7.54029574e-01, -2.51826241e-01,
         3.81491437e-05,  9.23325387e-06,  8.66707210e-06,
         6.22636174e-06],
       [-5.41754192e-01, -6.23955013e-01, -5.63189611e-01,
        -2.30465231e-06, -5.57796016e-07, -5.23592123e-07,
        

In [21]:
xtodunk(sol.x)

{'Ltot': array([6.02836122]),
 'Lx': array([0.32122647, 0.32919072, 0.34958281]),
 'Vtot': array([3.97163878]),
 'Vy': array([0.35170977, 0.33962121, 0.30866903]),
 'flashT': array([352.85497499])}

In [22]:
xtod(sol.x,d2)

{'FT': 450,
 'Ftot': 10,
 'Fz': array([0.33333333, 0.33333333, 0.33333333]),
 'Ltot': array([6.02836122]),
 'Lx': array([0.32122647, 0.32919072, 0.34958281]),
 'Vtot': array([3.97163878]),
 'Vy': array([0.35170977, 0.33962121, 0.30866903]),
 'flashP': 101325,
 'flashT': array([352.85497499])}